## A notebook to visualize data from the Letterboxed dataset to use in a Movie recomandation model

### Imports

In [4]:
!pip install scikit-surprise

  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-win_amd64.whl size=1291193 sha256=0e9923d5b7fcbfc3d7d77d18a3301448997971c0dc777215c35fba3ff6a4e9af
  Stored in directory: c:\users\aasmu\appdata\local\pip\cache\wheels\75\fa\bc\739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


In [6]:


import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import sklearn


from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split




import os

import numpy as np

from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt

import re

### Importing the files

In [15]:
#Erlend
movies = pd.read_csv("C:/Users/ranhe/PycharmProjects/MovieRecommendationML/CSVFiles/movie_data.csv", lineterminator='\n')
ratings = pd.read_csv("C:/Users/ranhe/PycharmProjects/MovieRecommendationML/CSVFiles/ratings_export.csv")
users = pd.read_csv("C:/Users/ranhe/PycharmProjects/MovieRecommendationML/CSVFiles/users_export.csv")

In [16]:
#Åsmund
movies = pd.read_csv("C:\\Users\\aasmu\\DAT158_MLProject\\MovieRecommendationML\\CSVFiles\\movie_data.csv", lineterminator='\n')
ratings = pd.read_csv("C:\\Users\\aasmu\\DAT158_MLProject\\MovieRecommendationML\\CSVFiles\\ratings_export.csv")
users = pd.read_csv("C:\\Users\\aasmu\\DAT158_MLProject\\MovieRecommendationML\\CSVFiles\\users_export.csv")

### Heading the data to see how it is setup 

In [17]:
movies.head(10)

,_id,genres,image_url,imdb_id,imdb_link,movie_id,movie_title,original_language,overview,popularity,production_countries,release_date,runtime,spoken_languages,tmdb_id,tmdb_link,vote_average,vote_count,year_released
0,5fc85f606758f69634496fd3,"[""Music"",""Animation""]",film-poster/4/6/4/4/4/0/464440-football-freaks...,NaN,NaN,football-freaks,Football Freaks,en,"Football crazy, football mad. Don’t watch this...",0.600,"[""United Kingdom""]",1971-12-05,0.0,[],535272.0,https://www.themoviedb.org/movie/535272/,0.0,0.0,1971.0
1,5fc85ff26758f696344ace0c,[],film-poster/2/4/5/5/0/0/245500-aftermath-0-230...,tt0586129,http://www.imdb.com/title/tt0586129/maindetails,aftermath-1960,Aftermath,en,Aftermath was the pilot for an unsold TV serie...,0.600,[],1960-04-17,22.0,[],318331.0,https://www.themoviedb.org/movie/318331/,8.0,1.0,1960.0
2,5fc85f606758f69634496fcd,"[""Drama""]",film-poster/9/3/3/1/8/93318-where-chimneys-are...,tt0045731,http://www.imdb.com/title/tt0045731/maindetails,where-chimneys-are-seen,Where Chimneys Are Seen,ja,Gosho’s most celebrated film both in Japan and...,1.568,"[""Japan""]",1953-03-05,108.0,"[""日本語""]",117779.0,https://www.themoviedb.org/movie/117779/,6.6,10.0,1953.0
3,5fc85f606758f69634496fd1,"[""Drama""]",NaN,tt0187327,http://www.imdb.com/title/tt0187327/maindetails,the-musicians-daughter,The Musician's Daughter,en,Carl Wagner's good wife was dying. His heart b...,0.600,"[""United States of America""]",1911-12-12,15.0,[],560377.0,https://www.themoviedb.org/movie/560377/,0.0,0.0,1911.0
4,5fc85f606758f69634496fd4,"[""Documentary""]",film-poster/4/5/4/6/0/3/454603-50-years-of-fab...,tt4769914,http://www.imdb.com/title/tt4769914/maindetails,50-years-of-fabulous,50 Years of Fabulous,en,50 Years of Fabulous recounts the rich history...,0.600,[],2018-05-17,75.0,[],525187.0,https://www.themoviedb.org/movie/525187/,0.0,0.0,2018.0
5,5fc85ff26758f696344ace3a,"[""Romance""]",film-poster/5/6/1/5/2/56152-where-love-has-gon...,tt0058745,http://www.imdb.com/title/tt0058745/maindetails,where-love-has-gone,Where Love Has Gone,en,A divorced couple's teen-age daughter stands t...,2.304,"[""United States of America""]",1964-11-02,111.0,"[""English""]",73348.0,https://www.themoviedb.org/movie/73348/,6.1,11.0,1964.0
6,5fc85ff26758f696344ace38,"[""Drama"",""Thriller""]",film-poster/4/7/8/4/4/5/478445-severance-0-230...,tt0096074,http://www.imdb.com/title/tt0096074/maindetails,severance-1988,Severance,en,"Ray Ponti is a down and out father, a man trap...",0.600,[],1988-01-01,95.0,[],549624.0,https://www.themoviedb.org/movie/549624/,0.0,0.0,1988.0
7,5fc85ff26758f696344ace53,[],NaN,tt9618868,http://www.imdb.com/title/tt9618868/maindetails,terra-di-transito,Terra di transito,it,NaN,0.600,"[""Italy""]",2014-09-11,0.0,[],419541.0,https://www.themoviedb.org/movie/419541/,5.0,2.0,2014.0
8,5fc85ff26758f696344ace64,"[""Music"",""Documentary""]",film-poster/1/1/2/2/5/4/112254-mc5-kick-out-th...,tt0477449,http://www.imdb.com/title/tt0477449/maindetails,mc5-kick-out-the-jams,MC5: Kick Out the Jams,en,Kick Out the Jams features many never before s...,0.600,[],1999-09-07,40.0,"[""English""]",148778.0,https://www.themoviedb.org/movie/148778/,9.0,1.0,1999.0
9,5fc85ff26758f696344ace6c,"[""Drama""]",film-poster/2/2/6/2/1/7/226217-sharing-the-sec...,tt0240894,http://www.imdb.com/title/tt0240894/maindetails,sharing-the-secret,Sharing the Secret,en,A teenage girl who feels she must always seem ...,3.531,"[""United States of America""]",2000-05-10,90.0,"[""English""]",299024.0,https://www.themoviedb.org/movie/299024/,6.9,10.0,2000.0


In [11]:
movies.shape

(285504, 19)

In [12]:
movies.isnull().sum()

_id                         0
genres                  10632
image_url               21097
imdb_id                 41702
imdb_link               41702
movie_id                    2
movie_title              2165
original_language       10632
overview                24256
popularity              10632
production_countries    10632
release_date            14454
runtime                 15449
spoken_languages        10632
tmdb_id                  5587
tmdb_link                5587
vote_average            10632
vote_count              10632
year_released            7572
dtype: int64

In [13]:
movies.describe()

,popularity,runtime,tmdb_id,vote_average,vote_count,year_released
count,274872.000000,270055.000000,279917.000000,274872.000000,274872.000000,277932.000000
mean,2.965787,66.386236,350433.142535,4.121326,61.627558,1992.439438
std,29.765843,142.899320,243316.810867,3.002508,547.102640,29.153126
min,0.000000,0.000000,2.000000,0.000000,0.000000,1874.000000
25%,0.600000,16.000000,134117.000000,0.000000,0.000000,1976.000000
50%,0.871000,80.000000,327033.000000,5.100000,2.000000,2005.000000
75%,2.004000,96.000000,522637.000000,6.400000,8.000000,2015.000000
max,6023.329000,51420.000000,952462.000000,10.000000,30211.000000,2028.000000


In [14]:
ratings.head(10)

,_id,movie_id,rating_val,user_id
0,5fc57c5d6758f6963451a07f,feast-2014,7,deathproof
1,5fc57c5d6758f6963451a063,loving-2016,7,deathproof
2,5fc57c5d6758f6963451a0ef,scripted-content,7,deathproof
3,5fc57c5d6758f6963451a060,the-future,4,deathproof
4,5fc57c5c6758f69634519398,mank,5,deathproof
5,5fc57c5d6758f6963451a05b,embers-2015,5,deathproof
6,5fc57c5d6758f6963451a172,the-social-network,10,deathproof
7,5fc57c5d6758f6963451a06c,miss-you-already,7,deathproof
8,5fc57c5d6758f6963451a0a0,saw-iii,7,deathproof
9,5fc57c5d6758f6963451a095,recess-schools-out,6,deathproof


In [15]:
users.head()

,_id,display_name,num_ratings_pages,num_reviews,username
0,5fc4172ec6cd28ebd99dd0e2,Lucy,32.0,1650,deathproof
1,5fc4172ec6cd28ebd99dd0ea,Matt Singer,52.0,1915,superpulse
2,5fc4172ec6cd28ebd99dd0ed,Sean Baker,21.0,1283,lilfilm
3,5fc4172ec6cd28ebd99dd0ee,iana,37.0,1177,ianamurray
4,5fc419171ebf67b9fbe48615,Lizzy,57.0,1810,punchdrunklizzy


In [16]:
ratings.shape

(11078167, 4)

### Cleaning data


In [18]:
# Removing duplicate rows
movies.drop_duplicates(inplace=True)
ratings.drop_duplicates(inplace=True)

# Removing missing values
movies.dropna(inplace=True)
ratings.dropna(inplace=True)

In [18]:
movies.isnull().sum()

_id                     0
genres                  0
image_url               0
imdb_id                 0
imdb_link               0
movie_id                0
movie_title             0
original_language       0
overview                0
popularity              0
production_countries    0
release_date            0
runtime                 0
spoken_languages        0
tmdb_id                 0
tmdb_link               0
vote_average            0
vote_count              0
year_released           0
dtype: int64

In [19]:
movies.head(100)

,_id,genres,image_url,imdb_id,imdb_link,movie_id,movie_title,original_language,overview,popularity,production_countries,release_date,runtime,spoken_languages,tmdb_id,tmdb_link,vote_average,vote_count,year_released
1,5fc85ff26758f696344ace0c,[],film-poster/2/4/5/5/0/0/245500-aftermath-0-230...,tt0586129,http://www.imdb.com/title/tt0586129/maindetails,aftermath-1960,Aftermath,en,Aftermath was the pilot for an unsold TV serie...,0.600,[],1960-04-17,22.0,[],318331.0,https://www.themoviedb.org/movie/318331/,8.0,1.0,1960.0
2,5fc85f606758f69634496fcd,"[""Drama""]",film-poster/9/3/3/1/8/93318-where-chimneys-are...,tt0045731,http://www.imdb.com/title/tt0045731/maindetails,where-chimneys-are-seen,Where Chimneys Are Seen,ja,Gosho’s most celebrated film both in Japan and...,1.568,"[""Japan""]",1953-03-05,108.0,"[""日本語""]",117779.0,https://www.themoviedb.org/movie/117779/,6.6,10.0,1953.0
4,5fc85f606758f69634496fd4,"[""Documentary""]",film-poster/4/5/4/6/0/3/454603-50-years-of-fab...,tt4769914,http://www.imdb.com/title/tt4769914/maindetails,50-years-of-fabulous,50 Years of Fabulous,en,50 Years of Fabulous recounts the rich history...,0.600,[],2018-05-17,75.0,[],525187.0,https://www.themoviedb.org/movie/525187/,0.0,0.0,2018.0
5,5fc85ff26758f696344ace3a,"[""Romance""]",film-poster/5/6/1/5/2/56152-where-love-has-gon...,tt0058745,http://www.imdb.com/title/tt0058745/maindetails,where-love-has-gone,Where Love Has Gone,en,A divorced couple's teen-age daughter stands t...,2.304,"[""United States of America""]",1964-11-02,111.0,"[""English""]",73348.0,https://www.themoviedb.org/movie/73348/,6.1,11.0,1964.0
6,5fc85ff26758f696344ace38,"[""Drama"",""Thriller""]",film-poster/4/7/8/4/4/5/478445-severance-0-230...,tt0096074,http://www.imdb.com/title/tt0096074/maindetails,severance-1988,Severance,en,"Ray Ponti is a down and out father, a man trap...",0.600,[],1988-01-01,95.0,[],549624.0,https://www.themoviedb.org/movie/549624/,0.0,0.0,1988.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,5fc85ff26758f696344acf71,"[""Comedy""]",film-poster/7/7/6/1/7761-boris-il-film-0-230-0...,tt1680058,http://www.imdb.com/title/tt1680058/maindetails,boris-il-film,Boris - Il film,it,A director and his crew attempt to make the tr...,5.449,"[""Italy""]",2011-04-01,108.0,"[""Italiano""]",58522.0,https://www.themoviedb.org/movie/58522/,6.9,434.0,2011.0
131,5fc85ff26758f696344acfdf,[],film-poster/4/4/7/0/0/5/447005-europe-after-th...,tt0077518,http://www.imdb.com/title/tt0077518/maindetails,europe-after-the-rain,Europe After the Rain,en,Dada came out of the craziness of World War On...,0.600,[],1978-01-01,88.0,[],517522.0,https://www.themoviedb.org/movie/517522/,0.0,0.0,1978.0
132,5fc85ff26758f696344acfed,"[""Mystery"",""Drama"",""Thriller""]",film-poster/5/5/4/7/5547-shadows-in-the-palace...,tt1194124,http://www.imdb.com/title/tt1194124/maindetails,shadows-in-the-palace,Shadows in the Palace,ko,Upon receiving the body of a maid in waiting w...,1.644,"[""South Korea""]",2007-10-18,112.0,"[""한국어/조선말""]",61128.0,https://www.themoviedb.org/movie/61128/,7.0,13.0,2007.0
133,5fc85ff26758f696344ad0db,"[""Animation"",""Action"",""Adventure""]",film-poster/3/1/5/1/3/31513-lupin-the-third-fa...,tt0170181,http://www.imdb.com/title/tt0170181/maindetails,lupin-the-third-farewell-to-nostradamus,Lupin the Third: Farewell to Nostradamus,ja,A simple diamond heist leads Lupin into the ma...,4.370,"[""Japan""]",1995-04-22,100.0,"[""日本語""]",31049.0,https://www.themoviedb.org/movie/31049/,6.7,37.0,1995.0


In [20]:
movies.shape

(214350, 19)

In [21]:
movies.columns

Index(['_id', 'genres', 'image_url', 'imdb_id', 'imdb_link', 'movie_id',
       'movie_title', 'original_language', 'overview', 'popularity',
       'production_countries', 'release_date', 'runtime', 'spoken_languages',
       'tmdb_id', 'tmdb_link', 'vote_average', 'vote_count', 'year_released'],
      dtype='object')

In [22]:
ratings.columns

Index(['_id', 'movie_id', 'rating_val', 'user_id'], dtype='object')

In [23]:
ratings.shape

(11078161, 4)

In [19]:
moviesTrimmed = movies.loc[:,["movie_id","movie_title","genres"]]

In [25]:
moviesTrimmed.head()

,movie_id,movie_title,genres
1,aftermath-1960,Aftermath,[]
2,where-chimneys-are-seen,Where Chimneys Are Seen,"[""Drama""]"
4,50-years-of-fabulous,50 Years of Fabulous,"[""Documentary""]"
5,where-love-has-gone,Where Love Has Gone,"[""Romance""]"
6,severance-1988,Severance,"[""Drama"",""Thriller""]"


In [22]:
mergedSet = pd.merge(moviesTrimmed, ratings)

In [27]:
mergedSet.shape

(10779876, 6)

In [28]:
mergedSet.head(100)

,movie_id,movie_title,genres,_id,rating_val,user_id
0,aftermath-1960,Aftermath,[],5fc58dd56758f69634c9c581,8,m_penalosa
1,where-chimneys-are-seen,Where Chimneys Are Seen,"[""Drama""]",5fc5815d6758f696346ffe97,9,vadekawa
2,where-chimneys-are-seen,Where Chimneys Are Seen,"[""Drama""]",5fc588a96758f69634a0dcd4,6,vrizov
3,where-chimneys-are-seen,Where Chimneys Are Seen,"[""Drama""]",5fc5912d6758f69634e05008,8,erich4chi
4,where-chimneys-are-seen,Where Chimneys Are Seen,"[""Drama""]",5fc592726758f69634e82a2c,7,swim
...,...,...,...,...,...,...
95,between-midnight-and-dawn,Between Midnight and Dawn,"[""Crime"",""Drama""]",60fc45a5d72bb4dd0a656fbd,6,dancedancedance
96,between-midnight-and-dawn,Between Midnight and Dawn,"[""Crime"",""Drama""]",60fc520bd72bb4dd0a698c87,5,acervofilmico
97,between-midnight-and-dawn,Between Midnight and Dawn,"[""Crime"",""Drama""]",61351cdb4385873fc4cbd703,7,axxonn
98,between-midnight-and-dawn,Between Midnight and Dawn,"[""Crime"",""Drama""]",613538f24385873fc4e1ee99,7,slig001


## Starting with building the model

In [34]:

# Define a reader with the appropriate scale for your ratings
reader = Reader(rating_scale=(1, 10))  # Adjust based on your rating scale

# Load the data from Pandas
data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating_val']], reader)

# Train-test split
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Use SVD for matrix factorization
algo = SVD()
algo.fit(trainset)

# Test on the testset
predictions = algo.test(testset)


AttributeError: 'DatasetAutoFolds' object has no attribute 'head'

In [28]:
from surprise import accuracy

# Compute and print RMSE
rmse = accuracy.rmse(predictions)

# Compute and print MAE
mae = accuracy.mae(predictions)


RMSE: 1.4063
MAE:  1.0657


In [32]:
import pickle

# save the iris classification model as a pickle file
model_pkl_file = "ml_movie_recommendation_model.pkl"

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(algo, file)

In [36]:
movies_simple = movies.loc[:,["movie_id","movie_title"]]
movies_simple.to_csv('movies_data_cleaned.csv', index=False)